# Transfer learning and fine-tuning with pretrained VGG16

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import math
from tensorflow import keras
from keras import layers
import numpy as np

The usual pipeline of a transfer-learning & fine-tuning workflow. See https://keras.io/guides/transfer_learning/.
1. Append trainable layers to a pretrained foundation model.
2. Freeze the base model, train the last layer until convergence.
3. Unfreeze the base model, train the whole model with very small learning rate.

## Data loading and preprocessing

In [2]:
data_train = tf.keras.utils.image_dataset_from_directory("Processed_Split/train", labels='inferred', image_size=(224, 224), batch_size=32)
data_val = tf.keras.utils.image_dataset_from_directory("Processed_Split/val", labels='inferred', image_size=(224, 224), batch_size=32)
data_test = tf.keras.utils.image_dataset_from_directory("Processed_Split/test", labels='inferred', image_size=(224, 224), batch_size=32)

Found 45828 files belonging to 50 classes.
Found 9438 files belonging to 50 classes.
Found 9504 files belonging to 50 classes.


## Data preprocessing

In [3]:
# Use the batch method to prepare batches.
data_train = data_train.map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))
data_val = data_val.map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))
data_test = data_test.map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))

## Model configuration

In [4]:
num_classes = 50

inputs = layers.Input(shape=(224, 224, 3))
vgg = keras.applications.vgg16.VGG16(include_top=False, weights="imagenet")
vgg.trainable = False
x = vgg(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = layers.Flatten()(x)
outputs = keras.layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics="accuracy")

## Model training (freeze ResNet)

In [5]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode="max", patience=4, verbose=1, baseline=0.0, restore_best_weights=True) # monitor validation loss, stop training if loss stops decreasing
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', mode="max", factor=0.5, patience=2, min_lr=0.000003125)

log = model.fit(x=data_train, epochs=20, validation_data=data_val, callbacks=[early_stopping, reduce_lr])

Epoch 1/20
1433/1433 [==============================] - 253s 172ms/step - loss: 0.6654 - accuracy: 0.8225 - val_loss: 0.3539 - val_accuracy: 0.8958 - lr: 0.0010
Epoch 2/20
1433/1433 [==============================] - 248s 173ms/step - loss: 0.2997 - accuracy: 0.9100 - val_loss: 0.2909 - val_accuracy: 0.9143 - lr: 0.0010
Epoch 3/20
1433/1433 [==============================] - 244s 170ms/step - loss: 0.2383 - accuracy: 0.9268 - val_loss: 0.2754 - val_accuracy: 0.9170 - lr: 0.0010
Epoch 4/20
1433/1433 [==============================] - 274s 191ms/step - loss: 0.2052 - accuracy: 0.9359 - val_loss: 0.2597 - val_accuracy: 0.9198 - lr: 0.0010
Epoch 5/20
1433/1433 [==============================] - 245s 171ms/step - loss: 0.1830 - accuracy: 0.9425 - val_loss: 0.2576 - val_accuracy: 0.9212 - lr: 0.0010
Epoch 6/20
1433/1433 [==============================] - 245s 170ms/step - loss: 0.1669 - accuracy: 0.9471 - val_loss: 0.2511 - val_accuracy: 0.9252 - lr: 0.0010
Epoch 7/20
1433/1433 [============

In [6]:
model.evaluate(data_test)
model.save("freeze_vgg1")

297/297 [==============================] - 42s 140ms/step - loss: 0.2285 - accuracy: 0.9314


INFO:tensorflow:Assets written to: freeze_vgg1/assets


INFO:tensorflow:Assets written to: freeze_vgg1/assets


In [7]:
data_test_fixed_order = tf.keras.utils.image_dataset_from_directory("Processed_Split/test", labels='inferred', image_size=(224, 224), batch_size=32, shuffle=False).map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))
y_pred = model.predict(data_test_fixed_order)
y_hat = np.argmax(y_pred, axis=1)
y_true = np.concatenate([y for x, y in data_test_fixed_order], axis=0)

Found 9504 files belonging to 50 classes.
297/297 [==============================] - 41s 139ms/step


In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print("Accuracy: ", accuracy_score(y_true, y_hat))
print("Precision: ", precision_score(y_true, y_hat, average='weighted'))
print("Recall: ", recall_score(y_true, y_hat, average='weighted'))
print("F1: ", f1_score(y_true, y_hat, average='weighted'))

Accuracy:  0.9313973063973064
Precision:  0.9308454485922624
Recall:  0.9313973063973064
F1:  0.9306989786257014


## Model training (unfreeze VGG)

In [9]:
vgg.trainable = True
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics="accuracy")

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode="max", patience=2, verbose=1, baseline=0.0, restore_best_weights=True) # monitor validation loss, stop training if loss stops decreasing
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', mode="max", factor=0.5, patience=1, min_lr=0.000003125)

log = model.fit(x=data_train, epochs=10, validation_data=data_val, callbacks=[early_stopping, reduce_lr])

Epoch 1/10
1433/1433 [==============================] - 623s 432ms/step - loss: 0.1550 - accuracy: 0.9490 - val_loss: 0.2006 - val_accuracy: 0.9378 - lr: 1.0000e-05
Epoch 2/10
1433/1433 [==============================] - 601s 419ms/step - loss: 0.0776 - accuracy: 0.9739 - val_loss: 0.1712 - val_accuracy: 0.9518 - lr: 1.0000e-05
Epoch 3/10
1433/1433 [==============================] - 588s 410ms/step - loss: 0.0561 - accuracy: 0.9817 - val_loss: 0.1893 - val_accuracy: 0.9482 - lr: 1.0000e-05
Epoch 4/10
1433/1433 [==============================] - 572s 399ms/step - loss: 0.0184 - accuracy: 0.9937 - val_loss: 0.1667 - val_accuracy: 0.9606 - lr: 5.0000e-06
Epoch 5/10
1433/1433 [==============================] - 558s 389ms/step - loss: 0.0119 - accuracy: 0.9962 - val_loss: 0.2238 - val_accuracy: 0.9527 - lr: 5.0000e-06
Epoch 6/10
1433/1433 [==============================] - 592s 413ms/step - loss: 0.0047 - accuracy: 0.9987 - val_loss: 0.1529 - val_accuracy: 0.9704 - lr: 3.1250e-06
Epoch 7/10

In [10]:
model.evaluate(data_test)
model.save('vgg1')

297/297 [==============================] - 46s 153ms/step - loss: 0.1546 - accuracy: 0.9680


INFO:tensorflow:Assets written to: vgg1/assets


INFO:tensorflow:Assets written to: vgg1/assets


## Evaluation metrics on test data

In [11]:
trained_model = tf.keras.models.load_model('vgg1')

In [12]:
data_test_fixed_order = tf.keras.utils.image_dataset_from_directory("Processed_Split/test", labels='inferred', image_size=(224, 224), batch_size=32, shuffle=False).map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))
y_pred = trained_model.predict(data_test_fixed_order)
y_hat = np.argmax(y_pred, axis=1)
y_true = np.concatenate([y for x, y in data_test_fixed_order], axis=0)

Found 9504 files belonging to 50 classes.
297/297 [==============================] - 45s 151ms/step


In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print("Accuracy: ", accuracy_score(y_true, y_hat))
print("Precision: ", precision_score(y_true, y_hat, average='weighted'))
print("Recall: ", recall_score(y_true, y_hat, average='weighted'))
print("F1: ", f1_score(y_true, y_hat, average='weighted'))

Accuracy:  0.968013468013468
Precision:  0.9680992533594471
Recall:  0.968013468013468
F1:  0.9677993098172192


In [14]:
import os
from sklearn.metrics import classification_report
print(classification_report(y_true, y_hat, digits=4, target_names=sorted(os.listdir("Processed_Split/test"))))

                                    precision    recall  f1-score   support

                 Amylax_triacantha     1.0000    0.7500    0.8571         4
           Aphanizomenon_flosaquae     0.9811    0.9924    0.9867      1049
       Aphanothece_paralleliformis     0.8333    1.0000    0.9091         5
                             Beads     1.0000    0.9500    0.9744        20
                      Centrales_sp     0.9254    0.8611    0.8921        72
             Ceratoneis_closterium     1.0000    0.7500    0.8571         8
                    Chaetoceros_sp     0.9849    0.9423    0.9631       208
             Chaetoceros_sp_single     0.9412    0.9697    0.9552        33
                    Chlorococcales     0.7333    0.7333    0.7333        15
                     Chroococcales     0.9333    0.6364    0.7568        22
                 Chroococcus_small     0.9597    0.9520    0.9558       125
                           Ciliata     0.8974    0.9459    0.9211        37
           